In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

# Load your dataset
data = pd.read_csv('train_delay.csv')

# Assuming 'delay' is the column representing train delays
# You may need to preprocess your data to extract features and preprocess them accordingly
# Here we're assuming 'X' contains features and 'y' contains the target delays
X = data.drop(columns=['delay', 'id']).values.reshape(-1, 1)  # Assuming 'id' is not a feature and reshaping to 2D
y = data['delay'].values

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape the input data to fit LSTM input shape (samples, time steps, features)
# Assuming you want to use 10 timesteps (you can adjust this based on your dataset)
timesteps = min(10, len(X_train))  # Adjust to the minimum of 10 or the length of your training data
n_features = 1  # Assuming univariate time series (only the 'delay' column)
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))  # Reshaping to (samples, 1, features)
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))  # Reshaping to (samples, 1, features)



# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model
loss = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {loss}')

# Predict train delays
predictions = model.predict(X_test_reshaped)

# You can further evaluate your predictions using metrics like MAE, RMSE, etc.


Epoch 1/50
3/3 [==============================] - 10s 835ms/step - loss: 15851.8496 - val_loss: 19655.4180
Epoch 2/50
3/3 [==============================] - 0s 23ms/step - loss: 15850.5146 - val_loss: 19653.5391
Epoch 3/50
3/3 [==============================] - 0s 21ms/step - loss: 15849.1074 - val_loss: 19651.5703
Epoch 4/50
3/3 [==============================] - 0s 21ms/step - loss: 15847.5000 - val_loss: 19649.3633
Epoch 5/50
3/3 [==============================] - 0s 21ms/step - loss: 15845.7842 - val_loss: 19646.8047
Epoch 6/50
3/3 [==============================] - 0s 20ms/step - loss: 15843.8965 - val_loss: 19643.8477
Epoch 7/50
3/3 [==============================] - 0s 27ms/step - loss: 15841.6533 - val_loss: 19640.4863
Epoch 8/50
3/3 [==============================] - 0s 30ms/step - loss: 15839.1299 - val_loss: 19636.6016
Epoch 9/50
3/3 [==============================] - 0s 21ms/step - loss: 15836.2500 - val_loss: 19632.0801
Epoch 10/50
3/3 [==============================] - 0s